In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/historical-flight-and-weather-data/12-2019.csv
/kaggle/input/historical-flight-and-weather-data/07-2019.csv
/kaggle/input/historical-flight-and-weather-data/05-2019.csv
/kaggle/input/historical-flight-and-weather-data/09-2019.csv
/kaggle/input/historical-flight-and-weather-data/06-2019.csv
/kaggle/input/historical-flight-and-weather-data/10-2019.csv
/kaggle/input/historical-flight-and-weather-data/08-2019.csv
/kaggle/input/historical-flight-and-weather-data/11-2019.csv


In [3]:
import glob
df = pd.concat([pd.read_csv(f) for f in glob.glob("/kaggle/input/historical-flight-and-weather-data/*.csv") ])

In [4]:
columns_to_use = [
    'origin_airport',
    'destination_airport',
    'actual_departure_dt',
    'actual_arrival_dt'
]

df = df[columns_to_use]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5512903 entries, 0 to 655103
Data columns (total 4 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   origin_airport       object
 1   destination_airport  object
 2   actual_departure_dt  object
 3   actual_arrival_dt    object
dtypes: object(4)
memory usage: 210.3+ MB


In [5]:
df.dropna(inplace= True)

In [6]:
def split_flight(flight):
    departure = [
        [flight.origin_airport,
        flight.actual_departure_dt]
    ]
    
    arrival = [
        [flight.destination_airport,
        flight.actual_arrival_dt]
    ]
    
    return (departure, arrival)

departures = []
arrivals = []
columns = [
    'airport',
    'date'
]
for flight in df.itertuples(index= False):
    departure, arrival = split_flight(flight)
    departures += departure
    arrivals += arrival
departures = pd.DataFrame(departures, columns= columns)
arrivals = pd.DataFrame(arrivals, columns= columns)

departures.info()
arrivals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5424261 entries, 0 to 5424260
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   airport  object
 1   date     object
dtypes: object(2)
memory usage: 82.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5424261 entries, 0 to 5424260
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   airport  object
 1   date     object
dtypes: object(2)
memory usage: 82.8+ MB


Ahora estandaricemos las fechas en minutos transcurridos desde la primera de las fechas

In [7]:
departures['date'] = pd.to_datetime(departures['date'])
arrivals['date'] = pd.to_datetime(arrivals['date'])
first_date = departures['date'].min()

Dividiremos los vuelos en intervalos de cinco minutos

In [8]:
min_date = departures['date'].min()
departures['date'] = (departures['date'] - min_date).apply(lambda x : int(x.total_seconds()//300))
arrivals['date'] = (arrivals['date'] - min_date).apply(lambda x : int(x.total_seconds()//300))

print(departures.head())
arrivals.head()

  airport   date
0     PHL  61918
1     PHL  61918
2     CLT  61641
3     PHX  61641
4     CLT  61635


,airport,date
0,ABE,61638
1,AVP,61639
2,BNA,61646
3,SBP,61653
4,MEM,61646


In [9]:
def transform_to_dictionary(df):
    airports = df['airport'].unique()
    max_date = df['date'].max()
    airport_date_dict = {airport : {} for airport in airports}
    for row in df.itertuples(index= False):
        airport_date_dict[row.airport][row.date] = airport_date_dict[row.airport].get(row.date, 0) + 1
    return airport_date_dict

departures = transform_to_dictionary(departures)
arrivals = transform_to_dictionary(arrivals)

In [11]:
import pickle
with open('airport_date_departure_saturation_dict.pkl', 'wb') as out_file:
    pickle.dump(departures, out_file)
with open('airport_date_arrivals_saturation_dict.pkl', 'wb') as out_file:
    pickle.dump(arrivals, out_file)